In [1]:
from pprint import pprint
from collections import ChainMap
from lark import Lark, InlineTransformer, Tree, Token
Tree._repr_html_ = lambda t: '<pre>%s</pre>' % t.pretty()
from runtime import eval
from parser import CscriptTransformer
from grammar import cscript
from symbol import Symbol

In [2]:
# Teste de gramática para operadores binários
value_sum = cscript.parse('1 + 2')
value_sub = cscript.parse('3 - 2')
value_mul = cscript.parse('12 * 3')
value_div = cscript.parse('14 / 2')
value_pow = cscript.parse('2 ^ 10')
soma = CscriptTransformer().transform(value_sum)
sub  = CscriptTransformer().transform(value_sub) 
mul = CscriptTransformer().transform(value_mul)
div = CscriptTransformer().transform(value_div)
pow_ = CscriptTransformer().transform(value_pow)

assert soma == [Symbol.ADD, 1.0, 2.0]
assert sub == [Symbol.SUB, 3.0, 2.0]
assert mul == [Symbol.MUL, 12.0, 3.0]
assert div == [Symbol.DIV, 14, 2]
assert pow_ == [Symbol.POW, 2, 10]

value_if = cscript.parse('if (T){2 + 2} else if (T){2 - 2}')
value_Eq = cscript.parse('if (2 == 2){1 + 2}')
value_Gt = cscript.parse('if (3 > 1){3 + 5}')
value_Le = cscript.parse('if (4 <= 5){2 + 5}')
value_Lt = cscript.parse('if (3 < 5){3 * 1}')
value_Ge = cscript.parse('if (3 >= 4){2 ^ 3}')
if_if = CscriptTransformer().transform(value_if)
if_Eq = CscriptTransformer().transform(value_Eq)
if_Gt = CscriptTransformer().transform(value_Gt)
if_Le = CscriptTransformer().transform(value_Le)
if_Lt = CscriptTransformer().transform(value_Lt)
if_Ge = CscriptTransformer().transform(value_Ge)


assert if_if == [Symbol.IF, True, [Symbol.ADD, 2.0, 2.0], [[True, [Symbol.SUB, 2.0, 2.0]]]]
assert if_Eq == [Symbol.IF, [Symbol.EQ, 2.0, 2.0], [Symbol.ADD, 1.0, 2.0]]
assert if_Gt == [Symbol.IF, [Symbol.GT, 3.0, 1.0], [Symbol.ADD, 3, 5]]
assert if_Le == [Symbol.IF, [Symbol.LE, 4.0, 5.0], [Symbol.ADD, 2, 5]]
assert if_Lt == [Symbol.IF, [Symbol.LT, 3.0, 5.0], [Symbol.MUL, 3, 1]]
assert if_Ge == [Symbol.IF, [Symbol.GE, 3.0, 4.0], [Symbol.POW, 2, 3]]

value_ternario = cscript.parse('asd = 3 > 0 ? 1 : 2')
if_ternario = CscriptTransformer().transform(value_ternario)


assert if_ternario == [Symbol.TERN, Symbol('asd'), [Symbol.GT, 3, 0], 1, 2]
eval(if_if, {})
#assert fat_func    ==[Symbol.FUNC, ['x', [Symbol.IF, [Symbol.LE, Symbol('x'), 1], [Symbol('return'), [1]], Symbol('return'), [Symbol.MUL, x, [Symbol('fat'), [Symbol.SUB, x, 1]]]]]]

4.0

In [3]:
assert eval(div, {}) == 7 
assert eval(soma, {}) == 3
assert eval(sub, {}) == 1
assert eval(mul, {}) == 36
assert eval(pow_, {}) == 1024


assert eval(if_if, {}) == 4
assert eval(if_Eq, {}) == 3
assert eval(if_Gt, {}) == 8
assert eval(if_Le, {}) == 7
assert eval(if_Lt, {}) == 3
assert eval(if_Ge, {}) is None

env = {'asd': 2}
assert eval(if_ternario, env) == None
assert env[Symbol('asd')] == 1

In [4]:

env = {Symbol('b'): 2}
print_ = cscript.parse('b = b == 2 ? 6 : 0')
print_eval = CscriptTransformer().transform(print_)
eval(print_eval, env)
assert env[Symbol('b')] == 6

In [5]:
func = cscript.parse('func lambda (a, b, c){a + b - c}')
call = cscript.parse('lambda(15, 94, 2)')
eval_func = CscriptTransformer().transform(func)
eval_call = CscriptTransformer().transform(call)
env = ChainMap()
assert eval(eval_func, env) is None
assert eval(eval_call, env) == 107

In [6]:
loop = cscript.parse("""
    a = 2;
  
    for(x = 3; x <= 10; x = x + 1){
        a = a + x
    };
    
    a
""")
loop_transform = CscriptTransformer().transform(loop)
print(loop_transform)
env = {}
assert eval(loop_transform, env) == 54
#assert env[1] == ChainMap({Symbol('x'): 11.0}, {Symbol('a'): 2.0, Symbol('x'): 3.0})

[block, [[=, a, 2.0], [for, [=, x, 3.0], [<=, x, 10.0], [=, x, [+, x, 1.0]], [=, a, [+, a, x]]], a]]


In [7]:
fat = cscript.parse(r"""
   // Função para calcular o fatorial
   func fat ( x ) { 
     if(x <= 1){
       1
     }
     else{
       x * fat( x - 1 )
     }
   };
   a = fat(7)
   a
""")
#at_call = cscript.parse('x = fat(5); x')

In [8]:
fat_eval = CscriptTransformer().transform(fat)
#call_eval = CscriptTransformer().transform(fat_call)

env = {}
print(fat_eval)
eval(fat_eval, env)
print(env)
#assert eval(fat_eval, env) is None
#print(env)
#print(eval(call_eval, env))
#assert eval(call_eval, env) == 120


[[block, [[func, fat, [x], [if, [<=, x, 1.0], 1.0, [[*, x, [call, fat, [[-, x, 1.0]]]]]]], [=, a, [call, fat, [7.0]]]]], a]
{fat: <function eval.<locals>.fn at 0x7f756ff83290>, a: 1.0}


In [12]:
while_p = cscript.parse("""
    a = 1;
    while(a < 10){
        a = a + 1
    }
""")

In [13]:
while_trans = CscriptTransformer().transform(while_p)

In [15]:
env = {}
eval(while_trans, env)
assert env[Symbol('a')] == 10

WHILE  None
WHILE  None
WHILE  None
WHILE  None
WHILE  None
WHILE  None
WHILE  None
WHILE  None
WHILE  None
